In [ ]:
import random
import time
import math
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial import KDTree
from scipy.stats import wasserstein_distance

import torch
import torch.nn as nn
from torch.nn import init
import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader
from torch.autograd.variable import Variable
from torch.utils.data import DataLoader

from geomloss import SamplesLoss
import os.path as osp
import torch_geometric.transforms as T
from torch_geometric.datasets import ModelNet
from torch_geometric.data import DataLoader

from IPython import embed
from tqdm import tqdm
import os

In [ ]:
# set_dist = []
# for i in range(100): 
#     x = torch.rand(500,1)
#     set_dist.append(x)
    
# for i in range(35): 
#     m = torch.distributions.beta.Beta(torch.tensor([.5]), torch.tensor([.5]))
#     x = m.sample([500])
#     set_dist.append(x)
    
# for i in range(35): 
#     m = torch.distributions.beta.Beta(torch.tensor([.7]), torch.tensor([.3]))
#     x = m.sample([500])
#     set_dist.append(x)
    
# for i in range(40): 
#     m = torch.distributions.beta.Beta(torch.tensor([.2]), torch.tensor([.7]))
#     x = m.sample([500])
#     set_dist.append(x)
    
# for i in range(50): 
#     m = torch.distributions.exponential.Exponential(torch.tensor([1.5]))
#     x = m.sample([500])
#     set_dist.append(x)
    
# for i in range(50): 
#     m = torch.distributions.exponential.Exponential(torch.tensor([2.5]))
#     x = m.sample([500])
#     set_dist.append(x)
    
# for i in range(100): 
#     m = torch.distributions.gamma.Gamma(torch.tensor([1.0]), torch.tensor([1.5]))
#     x = m.sample([500])
#     set_dist.append(x)
    
# for i in range(40): 
#     m = torch.distributions.laplace.Laplace(torch.tensor([1.0]), torch.tensor([1.5]))
#     x = m.sample([500])
#     set_dist.append(x)
    
# for i in range(59): 
#     m = torch.distributions.laplace.Laplace(torch.tensor([.5]), torch.tensor([1.0]))
#     x = m.sample([500])
#     set_dist.append(x)
    
# for i in range(100): 
#     m = torch.distributions.log_normal.LogNormal(torch.tensor([0.0]), torch.tensor([0.5]))
#     x = m.sample([500])
#     set_dist.append(x)   

    
# for i in range(59): 
#     m = torch.distributions.normal.Normal(torch.tensor([0.0]), torch.tensor([1.0]))
#     x = m.sample([500])
#     set_dist.append(x)   

# for i in range(50): 
#     m = torch.distributions.normal.Normal(torch.tensor([0.3]), torch.tensor([0.5]))
#     x = m.sample([500])
#     set_dist.append(x)   
    
    
# for i in range(100): 
#     m = torch.distributions.studentT.StudentT(torch.tensor([2.0]))
#     x = m.sample([500])
#     set_dist.append(x)      
    
# set_dist = torch.stack(set_dist)
# set_dist.shape

In [ ]:
path = osp.join('..', 'data/ModelNet10')
set_size = 100
batch_size = 16
data_ratio = 1
pre_transform, transform = T.NormalizeScale(), T.SamplePoints(set_size)
train_dataset = ModelNet(path, '10', True, transform, pre_transform)
data_size = train_dataset.__len__()
perm = torch.randperm(data_size)
idx = perm[:int(data_size*data_ratio)]
train_dataset = train_dataset.__getitem__(idx)
test_dataset = ModelNet(path, '10', False, transform, pre_transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=6)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=6)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

In [ ]:
# X = np.empty([len(train_loader.dataset), set_size, 3])
# Y = np.empty(len(train_loader.dataset), dtype=int)
    
# for i, batch in enumerate(train_loader):
#     n_data = batch.pos.reshape(-1, set_size, 3).to(device)
#     X[i * batch_size:(i + 1) * batch_size] = n_data.detach().cpu().numpy()
#     Y[i * batch_size:(i + 1) * batch_size] = batch.y.cpu().numpy()

# print(X.shape, Y.shape)

In [ ]:
# class SinkhornDistance(nn.Module):
#     r"""
#     Given two empirical measures each with :math:`P_1` locations
#     :math:`x\in\mathbb{R}^{D_1}` and :math:`P_2` locations :math:`y\in\mathbb{R}^{D_2}`,
#     outputs an approximation of the regularized OT cost for point clouds.
#     Args:
#         eps (float): regularization coefficient
#         max_iter (int): maximum number of Sinkhorn iterations
#         reduction (string, optional): Specifies the reduction to apply to the output:
#             'none' | 'mean' | 'sum'. 'none': no reduction will be applied,
#             'mean': the sum of the output will be divided by the number of
#             elements in the output, 'sum': the output will be summed. Default: 'none'
#     Shape:
#         - Input: :math:`(N, P_1, D_1)`, :math:`(N, P_2, D_2)`
#         - Output: :math:`(N)` or :math:`()`, depending on `reduction`
#     """
#     def __init__(self, eps, max_iter, reduction='none'):
#         super(SinkhornDistance, self).__init__()
#         self.eps = eps
#         self.max_iter = max_iter
#         self.reduction = reduction
# #         if device is None:
# #             self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# #         else:
# #             self.device = device
# #         self.to(self.device)

#     def forward(self, x, y):
#         # The Sinkhorn algorithm takes as input three variables :
#         C = self._cost_matrix(x, y)  # Wasserstein cost function
#         x_points = x.shape[-2]
#         y_points = y.shape[-2]
#         if x.dim() == 2:
#             batch_size = 1
#         else:
#             batch_size = x.shape[0]

#         # both marginals are fixed with equal weights
#         mu = torch.empty(batch_size, x_points, dtype=torch.float,
#                          requires_grad=False).fill_(1.0 / x_points).to(device).squeeze()
#         nu = torch.empty(batch_size, y_points, dtype=torch.float,
#                          requires_grad=False).fill_(1.0 / y_points).to(device).squeeze()

#         u = torch.zeros_like(mu).to(device)
#         v = torch.zeros_like(nu).to(device)
#         # To check if algorithm terminates because of threshold
#         # or max iterations reached
#         actual_nits = 0
#         # Stopping criterion
#         thresh = 1e-1

#         # Sinkhorn iterations
#         for i in range(self.max_iter):
#             u1 = u  # useful to check the update
#             u = self.eps * (torch.log(mu+1e-8) - torch.logsumexp(self.M(C, u, v), dim=-1)) + u
#             v = self.eps * (torch.log(nu+1e-8) - torch.logsumexp(self.M(C, u, v).transpose(-2, -1), dim=-1)) + v
#             err = (u - u1).abs().sum(-1).mean()

#             actual_nits += 1
#             if err.item() < thresh:
#                 break

#         U, V = u, v
#         # Transport plan pi = diag(a)*K*diag(b)
#         pi = torch.exp(self.M(C, U, V))
#         # Sinkhorn distance
#         cost = torch.sum(pi * C, dim=(-2, -1))

#         if self.reduction == 'mean':
#             cost = cost.mean()
#         elif self.reduction == 'sum':
#             cost = cost.sum()

#       #  return cost, pi, C
#         return cost

#     def M(self, C, u, v):
#         "Modified cost for logarithmic updates"
#         "$M_{ij} = (-c_{ij} + u_i + v_j) / \epsilon$"
#         return (-C + u.unsqueeze(-1) + v.unsqueeze(-2)) / self.eps

#     @staticmethod
#     def _cost_matrix(x, y, p=2):
#         "Returns the matrix of $|x_i-y_j|^p$."
#         x_col = x.unsqueeze(-2)
#         y_lin = y.unsqueeze(-3)
#         C = torch.sum((torch.abs(x_col - y_lin)) ** p, -1)
#         return C

#     @staticmethod
#     def ave(u, u1, tau):
#         "Barycenter subroutine, used by kinetic acceleration through extrapolation."
#         return tau * u + (1 - tau) * u1

In [ ]:
# def get_sinkhorn_distances(data, batch_size=20000, save_path=None, force_reload=False):
#     if force_reload is False and save_path is not None and os.path.exists(save_path):
#         distances = torch.load(save_path)
#         return distances
# #     sinkhorn = SinkhornDistance(eps=0.1, max_iter=100, device=device)
#     sinkhorn = SinkhornDistance(eps=0.1, max_iter=100)
# #     sinkhorn = SamplesLoss(loss="sinkhorn", scaling=0.7)
#     pairs = []
#     N = data.shape[0]
#     for i in tqdm(range(N)):
#         for j in range(i+1, N):
#             pairs.append(np.array([i,j]))
#     pairs = np.stack(pairs)
#     print(f'Len : {len(pairs)}')
#     print('Pairing is done. Start calculations.')
#     N_pairs = len(pairs)
# #     distances = torch.zeros(N, N, dtype=torch.float, device=device)
#     distances = torch.zeros(N, N, dtype=torch.double, device=device)
#     for batch_id in tqdm(range(math.ceil(N_pairs / batch_size))):
#         slice_ = slice(batch_id * batch_size, min(N_pairs, (batch_id+1) * batch_size))
#         idx_0 = pairs[slice_,0]
#         idx_1 = pairs[slice_,1]
# #         embed()
# #         slice_distances = sinkhorn(torch.tensor(data[idx_0,:,:], device=device), torch.tensor(data[idx_1,:,:], device=device)).detach()
#         slice_distances = sinkhorn(data[idx_0,:,:], data[idx_1,:,:]).detach()
#         distances[idx_0, idx_1] = slice_distances
#         distances[idx_1, idx_0] = slice_distances
#         # print('===pairs===')
#         # print(slice_)
#         # print(slice_distances.shape)
#         # for i in range(slice_.start, slice_.stop):
#         #   print(f'{pairs[i,:]} -> {slice_distances[i - slice_.start]}')
#     if save_path is not None and os.path.exists(save_path):
#         save_path.parent.mkdir(exist_ok=True)
#         torch.save(distances, save_path)
#     return distances

In [ ]:
# # out = get_sinkhorn_distances(set_dist.to(device), batch_size=1024)
# out = get_sinkhorn_distances(torch.tensor(X, device=device), batch_size=512)
# torch.save(out, "distance_matrix_512_arijit")

In [ ]:
# distances = torch.load("distance_matrix_512_arijit")

In [ ]:
# import os
# from sklearn.manifold import TSNE
# from matplotlib import pyplot as plt

# tsne = TSNE(n_components=2, init='random', metric='precomputed')
# # tsne_embedding = tsne.fit_transform(out.cpu())
# # plt.scatter(tsne_embedding[:,0], tsne_embedding[:,1], label='T-SNE')
# # plt.legend()

# X_2d = tsne.fit_transform(distances.cpu())
# target_ids = range(10)
# plt.figure(figsize=(6, 5))
# colors = 'r', 'g', 'b', 'c', 'm', 'y', 'k', 'brown', 'orange', 'purple'
# for i, c in zip(target_ids, colors):
#     plt.scatter(X_2d[Y == i, 0], X_2d[Y == i, 1], c=c, label=i)
# plt.legend()
# # plt.savefig("tsne/test_%d.png" %t)

In [ ]:
# import umap
# umap_embedder = umap.UMAP(metric='precomputed')
# # umap_embedding = umap_embedder.fit_transform(out.cpu())
# # plt.scatter(umap_embedding[:,0], umap_embedding[:,1], label='UMAP')
# # plt.legend()

# X_2d = umap_embedder.fit_transform(out.cpu())
# target_ids = range(10)
# plt.figure(figsize=(6, 5))
# colors = 'r', 'g', 'b', 'c', 'm', 'y', 'k', 'brown', 'orange', 'purple'
# for i, c in zip(target_ids, colors):
#     plt.scatter(X_2d[Y == i, 0], X_2d[Y == i, 1], c=c, label=i)
# plt.legend()

In [ ]:
# import phate

# phate_operator = phate.PHATE(knn=5, decay=20, t=150, knn_dist='precomputed', n_jobs=-1)
# # phate_embedding = phate_operator.fit_transform(out.cpu())
# # plt.scatter(phate_embedding[:,0], phate_embedding[:,1], label='PHATE')
# # plt.legend()
# X_2d = phate_operator.fit_transform(out.cpu())
# target_ids = range(10)
# plt.figure(figsize=(6, 5))
# colors = 'r', 'g', 'b', 'c', 'm', 'y', 'k', 'brown', 'orange', 'purple'
# for i, c in zip(target_ids, colors):
#     plt.scatter(X_2d[Y == i, 0], X_2d[Y == i, 1], c=c, label=i)
# plt.legend()

In [ ]:
class DeepSet(nn.Module):

    def __init__(self, in_features, set_features):
        super(DeepSet, self).__init__()
        self.in_features = in_features
        self.out_features = set_features
        self.feature_extractor = nn.Sequential(
            nn.Linear(in_features, 50),
            nn.ELU(inplace=True),
            nn.Linear(50, 100),
            nn.ELU(inplace=True),
            nn.Linear(100, set_features),
        )

        self.regressor = nn.Sequential(
            nn.Linear(set_features, 30),
            nn.ELU(inplace=True),
            nn.Linear(30, 30),
            nn.ELU(inplace=True),
            nn.Linear(30, 10),
            nn.ELU(inplace=True),
            nn.Linear(10, 2),
        )
        
        
    def forward(self, input):
        x = input
        x = self.feature_extractor(x)
        x = x.sum(dim=1)
#         x = x.mean(dim=1)
#         x = x.max(dim=1).values
        x = self.regressor(x)
        return x

class PermEqui1_max(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(PermEqui1_max, self).__init__()
        self.Gamma = nn.Linear(in_dim, out_dim)

    def forward(self, x):
        xm, _ = x.max(1, keepdim=True)
        x = self.Gamma(x-xm)
        return x
    
class DeepSet_pc(nn.Module):

    def __init__(self, d_dim=256, x_dim=3):
        super(DeepSet_pc, self).__init__()
        self.d_dim = d_dim
        self.x_dim = x_dim

        self.phi = nn.Sequential(
          PermEqui1_max(self.x_dim, self.d_dim),
          nn.Tanh(),
          PermEqui1_max(self.d_dim, self.d_dim),
          nn.Tanh(),
          PermEqui1_max(self.d_dim, self.d_dim),
          nn.Tanh(),
        )

        self.ro = nn.Sequential(
           nn.Dropout(p=0.5),
           nn.Linear(self.d_dim, self.d_dim),
#            nn.Tanh(),
#            nn.Dropout(p=0.5),
#            nn.Linear(self.d_dim, 40),
#             nn.Linear(self.d_dim, self.d_dim),
        )
        print(self)

    def forward(self, x):
        phi_output = self.phi(x)
        sum_output, _ = phi_output.max(1)
        ro_output = self.ro(sum_output)
        return ro_output

In [ ]:
# sinkhorn = SinkhornDistance(eps=0.1, max_iter=100, reduction=None).to(device)
# sinkhorn = SinkhornDistance(eps=0.1, max_iter=100).to(device)
# sinkhorn = SamplesLoss(loss="sinkhorn", p=2, blur=.05)
# sinkhorn = SamplesLoss(loss="sinkhorn", scaling=0.7, debias=False)
sinkhorn = SamplesLoss(loss="sinkhorn", p=1, scaling=0.9, debias=False)

In [ ]:
# class MyDataset(Dataset):
#     def __init__(self, data, transform=None):
#         self.data = data.float()
        
#         self.transform = transform

#     def __getitem__(self, index):
#         x = self.data[index]
        
#         if self.transform:
#             x = self.transform(x)
           
#         return x

#     def __len__(self):
#         return len(self.data)

In [ ]:
# dataset = MyDataset(set_dist)
# train_loader = DataLoader(dataset, batch_size = 12, shuffle = True)

In [ ]:
# model = DeepSet(1, 36).to(device)
# model = DeepSet(3, 10).to(device)
dim=256
model = DeepSet_pc(d_dim=dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# checkpoint = torch.load('deepset_dist_2conditions_5.pt')
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# loss = checkpoint['loss']

model.train()

In [ ]:
import os
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
os.makedirs("tsne/", exist_ok=True)
tsne = TSNE(n_components=2, random_state=0)

Wasserstein distance has the following properties: 
1) W(aX,aY) = |a|W(X,Y)
2) W(X+x, Y+x) = W(X,Y)
3) ...
Here we remove the 3rd condition.

In [ ]:
from scipy.stats import pearsonr

num_epochs = 5
num_paths = 100
running_loss = []
total = 0
total_time = []
total_corr = []
for t in range(num_epochs):
    start = time.time()
    total_loss = 0
    sink1 = []
    euc = []
    for n_batch, batch in enumerate(train_loader):
        
        batch = batch.pos.reshape(-1, set_size, 3)
        n_data = Variable(batch.to(device), requires_grad=True)
#         n_data = batch.to(device)
        batch_size = n_data.shape[0]
#         a = torch.rand(1).to(device)
#         b = torch.rand(1).to(device)
       
    
        optimizer.zero_grad()
        y = model(n_data)
#         y_a = model(a*n_data)
#         y_translate = model(n_data + b)
        
#         loss = 0
        src = np.random.randint(0, batch_size, size=num_paths)
        dest = np.random.randint(0, batch_size, size=num_paths)
        sink_dist = sinkhorn(n_data[src],n_data[dest])
        sink_dist [src == dest] = 0
        euc_dist = torch.norm(y[src]-y[dest], p=2, dim=1)
        loss = torch.norm(sink_dist-euc_dist, p=2)/num_paths
        
#         for i in range(len(batch)):
#             for j in range(i+1,len(batch)):
#                 y_ij = torch.norm(y[i]-y[j], p=2)
#                 w_ij = sinkhorn(n_data[i],n_data[j])
                
#                 ya_ij = torch.norm(y_a[i]-y_a[j], p=2)
#                 y_translate_ij = torch.norm(y_translate[i]-y_translate[j], p=2)
                
#                 diff_translate_ij = torch.norm(y_translate[i]-y[j], p=2)**2
                   
#                 loss += torch.norm(y_ij-w_ij, p=2) + (ya_ij-a*y_ij)**2 + (y_translate_ij- y_ij)**2
                
#                 del w_ij
            
        #Trying without the last condition. That condition seems buggy
        
#         loss = calculate_loss(batch, n_data, a, y, y_a, y_translate)
        
        
#         loss = loss/(len(batch)*(len(batch)-1)/2)
       
        loss.backward()
       
        optimizer.step()
        
#         print("Batch %d Done", n_batch)
        total_loss += loss
    
        sink1.extend(sink_dist[src != dest].cpu().detach().numpy())
        euc.extend(euc_dist[src != dest].cpu().detach().numpy())
#         cor, _ = pearsonr(sink_dist[src != dest].cpu().detach().numpy(),euc_dist[src != dest].cpu().detach().numpy())
#         corr.append(cor)
    
        
    running_loss.append(total_loss)
    print(total_loss)
    
    end = time.time()
    total += end-start
    total_time.append(total)
    
    corr, _ = pearsonr(sink1,euc)
    total_corr.append(corr)

model.eval()
with torch.no_grad():

    X = np.empty([len(train_loader.dataset), 256])
    Y = np.empty(len(train_loader.dataset), dtype=int)

    start = 0
    for i, batch in enumerate(train_loader):
        n_data = batch.pos.reshape(-1, set_size, 3).to(device)
        encodings = model(n_data)
        batch_size = n_data.shape[0]
        X[start:start+batch_size] = encodings.detach().cpu().numpy()
        Y[start:start+batch_size] = batch.y.cpu().numpy()
        start += batch_size

    X_2d = tsne.fit_transform(X)
    target_ids = range(10)

    plt.figure(figsize=(6, 5))
    colors = 'r', 'g', 'b', 'c', 'm', 'y', 'k', 'brown', 'orange', 'purple'
    for i, c, label in zip(target_ids, colors, target_ids):
        plt.scatter(X_2d[Y == i, 0], X_2d[Y == i, 1], c=c, label=label)
    plt.legend()
    plt.savefig("tsne/test_%d.png" %t)

In [ ]:
plt.plot(total_time, total_corr)

In [ ]:
len(running_loss)

In [ ]:
running_loss.index(min(running_loss))

In [ ]:
#First save is at 31 epochs, 2nd save after 31 epochs, 3rd save is after 41 epochs, 
#4th save is after 16 epochs, 5th save is after 35+28+36 +27+27+7epochs
# torch.save(model.state_dict(),'deepset_dist_2conditions_5.pkl')
torch.save(model.state_dict(),'pc_10_100_epoch100.pkl')

In [ ]:
#First save is at 31 epochs 
torch.save({
    
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
            
            }, 'pc_10_100_epoch100.pt')

In [ ]:
checkpoint = torch.load('pc_10_100_epoch100.pt')
model.load_state_dict(checkpoint['model_state_dict'])
# checkpoint = torch.load('pc_10_1000_w1_50.pkl')
# model.load_state_dict(checkpoint)

In [ ]:
L = nn.CrossEntropyLoss().cuda()

def train(epoch, model, optimizer):
    model.train()
    total_loss = 0

    for data in train_loader:
        data = data.to(device)
        batch = data.pos.reshape(-1, set_size, 3)
        optimizer.zero_grad()
        loss = L(model(batch), data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss
    print(total_loss)

In [ ]:
def test(loader, model):
    model.eval()

    correct = 0
    for data in loader:
        data = data.to(device)
        batch = data.pos.reshape(-1, set_size, 3)
        with torch.no_grad():
            pred = model(batch).max(1)[1]
        correct += pred.eq(data.y).sum().item()
    return correct / len(loader.dataset)

In [ ]:
class pc_classification_layers_10(nn.Module):
    def __init__(self, inp_dim=256, out_dim=10):
        super(pc_classification_layers_10, self).__init__()
        self.fc = nn.Sequential(
#             nn.Tanh(),
#             nn.Dropout(p=0.5),
#             nn.Linear(inp_dim, 64),
#             nn.Tanh(),
#             nn.Dropout(p=0.5),
#             nn.Linear(64, out_dim),
            nn.Tanh(),
            nn.Dropout(p=0.5),
            nn.Linear(inp_dim, out_dim),
        )
        
    def forward(self, x):
        x = self.fc(x)
        return x
    
class pc_classification_layers_40(nn.Module):
    def __init__(self, inp_dim=256, out_dim=40):
        super(pc_classification_layers_40, self).__init__()
        self.fc = nn.Sequential(
#             nn.Tanh(),
#             nn.Dropout(p=0.5),
#             nn.Linear(inp_dim, 128),
#             nn.Tanh(),
#             nn.Dropout(p=0.5),
#             nn.Linear(128, 40),
            nn.Tanh(),
            nn.Dropout(p=0.5),
            nn.Linear(inp_dim, out_dim),
        )
        
    def forward(self, x):
        x = self.fc(x)
        return x

In [ ]:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
model_new = nn.Sequential(model, pc_classification_layers_10()).to(device)
# model_new = nn.Sequential(model, pc_classification_layers_40()).to(device)

In [ ]:
optimizer_new = torch.optim.Adam(model_new.parameters(), lr=0.001)

In [ ]:
num_epochs = 20
for epoch in range(num_epochs):
        train(epoch, model_new, optimizer_new)
        test_acc = test(test_loader, model_new)
        print('Epoch: {:03d}, Test: {:.4f}'.format(epoch, test_acc))

In [ ]:
def plot_tsne(loader, model, out_dim, epoch=0):
    
    model.eval()
    with torch.no_grad():
    
        X = np.empty([len(loader.dataset), out_dim])
        Y = np.empty(len(loader.dataset), dtype=int)

        start = 0
        for i, batch in enumerate(loader):
            n_data = batch.pos.reshape(-1, set_size, 3).to(device)
            encodings = model(n_data)
            batch_size = n_data.shape[0]
            X[start:start+batch_size] = encodings.detach().cpu().numpy()
            Y[start:start+batch_size] = batch.y.cpu().numpy()
            start += batch_size

        X_2d = tsne.fit_transform(X)
        target_ids = range(10)

        plt.figure(figsize=(6, 5))
        colors = 'r', 'g', 'b', 'c', 'm', 'y', 'k', 'brown', 'orange', 'purple'
        for i, c, label in zip(target_ids, colors, target_ids):
            plt.scatter(X_2d[Y == i, 0], X_2d[Y == i, 1], c=c, label=label)
        plt.legend()
        plt.savefig("tsne/morelayers_%d.png" %epoch)

In [ ]:
plot_tsne(train_loader, model_new, 10)
# plot_tsne(train_loader, model_new, 40)

In [ ]:
for param in model_new.parameters():
    param.requires_grad = True
optimizer_full = torch.optim.Adam(model_new.parameters(), lr=0.0001)

In [ ]:
num_epochs = 20
for epoch in range(num_epochs):
    train(epoch, model_new, optimizer_full)
            
    test_acc = test(test_loader, model_new)
    print('Epoch: {:03d}, Test: {:.4f}'.format(epoch, test_acc))

In [ ]:
plot_tsne(train_loader, model_new, 10)
# plot_tsne(train_loader, model_new, 40)

In [ ]:
class DeepSet_pc_full_10(nn.Module):

    def __init__(self, d_dim=256, x_dim=3):
        super(DeepSet_pc_full_10, self).__init__()
        self.d_dim = d_dim
        self.x_dim = x_dim

        self.phi = nn.Sequential(
          PermEqui1_max(self.x_dim, self.d_dim),
          nn.Tanh(),
          PermEqui1_max(self.d_dim, self.d_dim),
          nn.Tanh(),
          PermEqui1_max(self.d_dim, self.d_dim),
          nn.Tanh(),
        )

        self.ro = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(self.d_dim, self.d_dim),
#             nn.Tanh(),
#             nn.Dropout(p=0.5),
#             nn.Linear(self.d_dim, 64),
#             nn.Tanh(),
#             nn.Dropout(p=0.5),
#             nn.Linear(64, 10),
            nn.Tanh(),
            nn.Dropout(p=0.5),
            nn.Linear(self.d_dim, 10),
        )
        print(self)

    def forward(self, x):
        phi_output = self.phi(x)
        sum_output, _ = phi_output.max(1)
        ro_output = self.ro(sum_output)
        return ro_output
    
class DeepSet_pc_full_40(nn.Module):

    def __init__(self, d_dim=256, x_dim=3):
        super(DeepSet_pc_full_40, self).__init__()
        self.d_dim = d_dim
        self.x_dim = x_dim

        self.phi = nn.Sequential(
          PermEqui1_max(self.x_dim, self.d_dim),
          nn.Tanh(),
          PermEqui1_max(self.d_dim, self.d_dim),
          nn.Tanh(),
          PermEqui1_max(self.d_dim, self.d_dim),
          nn.Tanh(),
        )

        self.ro = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(self.d_dim, self.d_dim),
            nn.Tanh(),
            nn.Dropout(p=0.5),
            nn.Linear(self.d_dim, 128),
            nn.Tanh(),
            nn.Dropout(p=0.5),
            nn.Linear(128, 40),
        )
        print(self)

    def forward(self, x):
        phi_output = self.phi(x)
        sum_output, _ = phi_output.max(1)
        ro_output = self.ro(sum_output)
        return ro_output

In [ ]:
dim=256
model_base = DeepSet_pc_full_10(d_dim=dim).to(device)
# model_base = DeepSet_pc_full_40(d_dim=dim).to(device)

In [ ]:
optimizer_base = torch.optim.Adam(model_base.parameters(), lr=0.0001)

In [ ]:
num_epochs = 20
for epoch in range(num_epochs):
        train(epoch, model_base, optimizer_base)
        test_acc = test(test_loader, model_base)
        print('Epoch: {:03d}, Test: {:.4f}'.format(epoch, test_acc))

In [ ]:
plot_tsne(train_loader, model_base, 10)
# plot_tsne(train_loader, model_base, 40)

In [ ]:
x=torch.rand(1,500,1)
y = torch.rand(1,500,1)

In [ ]:
model(x)

In [ ]:
model(y)

In [ ]:
m = torch.distributions.normal.Normal(torch.tensor([0.0]), torch.tensor([1.0]))
m1 = m.sample([500]).view(1,-1,1)

In [ ]:
model(m1)

In [ ]:
m2 = m.sample([500]).view(1,-1,1)

In [ ]:
model(m2)

In [ ]:
n = torch.distributions.normal.Normal(torch.tensor([.05]), torch.tensor([1.0]))

In [ ]:
n1 = n.sample([500]).view(1,-1,1)

In [ ]:
model(n1)

In [ ]:
n2 = n.sample([500]).view(1,-1,1)

In [ ]:
model(n2)

In [ ]:
N = torch.distributions.normal.Normal(torch.tensor([.5]), torch.tensor([.75]))
N1 = N.sample([500]).view(1,-1,1)
N2 = N.sample([500]).view(1,-1,1)

In [ ]:
model(N1)

In [ ]:
model(N2)

In [ ]:
A = torch.ones(500).view(1,-1,1)
B = torch.zeros(500).view(1,-1,1)

In [ ]:
model(A)

In [ ]:
model(B)

In [ ]:
sinkhorn(A,B)

In [ ]:
C = torch.distributions.bernoulli.Bernoulli(torch.tensor([.3]))
D = torch.distributions.bernoulli.Bernoulli(torch.tensor([.5]))

In [ ]:
C1 = C.sample([500]).view(1,-1,1)
D1 = D.sample([500]).view(1,-1,1)

In [ ]:
model(C1)

In [ ]:
model(D1)

In [ ]:
sinkhorn(C1,D1)